# Mapping the 2021 La Palma *Cumbre Vieja* Eruption

In [4]:
import matplotlib.pyplot as plt
from ipywidgets import interact

import lplm_geoprocessing as lpgeo
import lplm_io as lpio
import lplm_presentation as lpp
import lplm_utils

In [72]:
# TODO: remove when not needed
import importlib
importlib.reload(lpgeo)
importlib.reload(lpio)
importlib.reload(lpp)
importlib.reload(lplm_utils)

<module 'lplm_utils' from '/home/jovyan/lplm/lplm_utils.py'>

Start our journey...

In [3]:
# Center needs to be passed in as EPSG:4326 regardless of desired map CRS. 
map_center = ( (28.58 + 28.65) / 2, (-17.97 + -17.84) / 2)
 
from sidecar import Sidecar
from ipyleaflet import Map, LayersControl, projections
lp_map = Map(
    center=map_center,
    zoom=13,
    crs=projections.EPSG3857,
    scroll_wheel_zoom=True)
lp_map.add_control(LayersControl())
sc = Sidecar()
with sc:
    display(lp_map)

Load the image stack.

In [4]:
xds_grd = lpio.load_series("data")

## ❔ Choose an image

Inspect the images, and choose one to perform the analysis on.

In [5]:
plt.rcParams["figure.figsize"] = (12,8)
plt.rcParams["figure.dpi"] = 100

def view_scene(date):
    # Use this date for the rest of the notebook
    global selected_date 
    selected_date = date
    
    xds_grd.sel(date=date).plot(cmap="gray")
    
interact(view_scene, date=lpp.get_date_labels(xds_grd))

interactive(children=(Dropdown(description='date', options=(('2021-09-10', numpy.datetime64('2021-09-10T00:00:…

<function __main__.view_scene(date)>

Add this to the map.

In [6]:
from ipyleaflet import ImageOverlay
import base64

layer_sar = lpp.make_sar_layer(xds_grd, selected_date)
lp_map.add_layer(layer_sar)

Warning 1: +init=epsg:XXXX syntax is deprecated. It might return a CRS with a non-EPSG compliant axis order.


## Segmentation

SLIC

In [7]:
lpgeo.apply_segmentation(xds_grd, selected_date)

In [8]:
gdf = lpgeo.vectorize_segments(xds_grd)

Display the result.

In [9]:
# This may take a little while.
from ipyleaflet import GeoData
# Have to convert to EPSG:4326 first, despite map CRS.
layer_seg = GeoData(
    geo_dataframe = gdf.to_crs("EPSG:4326"),
    style = { "fillOpacity": 0.0, "weight": 1.0 },
    name = "Segments")
lp_map.add_layer(layer_seg)

## Analysis, Part 1
Calculate statistical features of segments, and from those derive initial lava-likeness metrics.

In [10]:
xds_segstats = lpgeo.segment_stats(xds_grd)

In [11]:
lpgeo.enrich(gdf, xds_segstats, selected_date)

In [12]:
from ipyleaflet import Choropleth
from branca.colormap import linear
import json

# lp_map.remove(layer_seg)

layer_enriched = lpp.make_feature_layer(gdf, "local_lava_likeness")

lp_map.add_layer(layer_enriched)


## Analysis, Part 2
Now we apply some neighbourhood-based operations to estimate the lava field.

### ❔ Choose a starting point

Choose a starting point to seed the region-growing algorithm - ideally somewhere clearly in the lava field.

In [13]:
from ipyleaflet import Marker
marker = Marker(location=map_center, draggable=True, name="Start")
lp_map.add_layer(marker)

Find the matching segment.

In [14]:
from pyproj import Transformer
t = Transformer.from_crs("epsg:4326", "epsg:3857")
start_x, start_y = t.transform(marker.location[0], marker.location[1])
start_segment_id = lpgeo.get_segment_id(gdf, start_x, start_y)

Now, estimate the lava region.

In [15]:
# TODO: Fix warnings for this:
import warnings
warnings.filterwarnings('ignore')
lpgeo.lava_likeness_overall(gdf, start_segment_id)

gdf_lava_est = lpgeo.extract_lava_region(gdf, start_segment_id)

Show the results.

In [16]:
layer_neigh_ll = lpp.make_feature_layer(gdf, "neighbourhood_lava_likeness")

layer_lava = GeoData(
    geo_dataframe = gdf_lava_est.to_crs("EPSG:4326"),
    style = { "fillOpacity": 1.0, "weight": 1.0, "fillColor": "orange" },
    name = "Estimated lava field")

lp_map.add_layer(layer_neigh_ll)
lp_map.add_layer(layer_lava)

The Copernicus Emergency Rapid Mapping service provided maps through the course of the eruption. These were created with the aid of high resolution sources of various types, such as [COSMO-SkyMed](https://earth.esa.int/eogateway/missions/cosmo-skymed), a high resolution X-band SAR mission. See the [event page](https://emergency.copernicus.eu/mapping/ems/volcano-eruption-la-palma-spain) for further details. With a few of these as a reference, let's compare our results... 

In [82]:
ems_maps = {
    "2021-11-15": geopandas.read_file("ems-reference/EMSR546_AOI01_GRA_MONIT52_observedEventA_r1_v1.shp"),
    "2021-11-21": geopandas.read_file("ems-reference/EMSR546_AOI01_GRA_MONIT53_observedEventA_r1_v1.shp"),
    "2021-12-18": geopandas.read_file("ems-reference/EMSR546_AOI01_GRA_MONIT63_observedEventA_r1_v1.shp"),
}

def ems_layer_name(key):
    return f"EMS Rapid Mapping @ {key}"

def make_ems_layer(key):
    layer = GeoData(
        geo_dataframe = ems_maps[key].to_crs("EPSG:4326"),
        style = { "fillOpacity": 0.0, "weight": 1.0, "color": "red" },
        name = ems_layer_name(key))
    return layer

layers_ems = {}

for key in ems_maps.keys():
    layers_ems[key] = make_ems_layer(key)

def show_ems_layer(ems_map_date):
    global selected_ems
    existing = lpp.find_layers(lp_map, ems_layer_name(selected_ems))
    for l in existing:
        lp_map.remove(l)
    lp_map.add_layer(layers_ems[ems_map_date])
    selected_ems = ems_map_date
    return None
    
interact(show_ems_layer, ems_map_date=ems_maps.keys())

interactive(children=(Dropdown(description='ems_map_date', options=('2021-11-15', '2021-11-21', '2021-12-18'),…

<function __main__.show_ems_layer(ems_map_date)>